In [1]:
# Import the pipeline function
from transformers import pipeline
from pydub import AudioSegment
from dotenv import load_dotenv

import os

# Load environment variables from .env file
load_dotenv("/data/EES/SoroushAI/.env")

# Print to verify
print("Hugging Face cache directory:", os.getenv("HF_HUB_CACHE"))

/data/soroush/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hugging Face cache directory: /data/huggingface


In [2]:
# Load the MP3 file
audio = AudioSegment.from_mp3("Masnavi_D6_J79_20240707.mp3")

# Define the duration for 10 minutes (in milliseconds)
ten_minutes_ms = 10 * 60 * 1000

# Extract desired segment
first_10_minutes = audio[3 * ten_minutes_ms: 4 * ten_minutes_ms]

# Export the segment to a new file
first_10_minutes.export("segment.wav", format="wav")

# Define the path to the extracted audio file
audio_file = "segment.wav"

### Whisper v3

In [14]:
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

generate_kwargs = {
    "language": "fa",
    "return_timestamps": True}

Device set to use cuda:0


In [15]:
result = pipe(audio_file, generate_kwargs=generate_kwargs)

/data/soroush/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=fa, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=fa.


In [16]:
# Extract the transcribed text
transcribed_text = result["text"]

# Print the output
print("Transcribed Text:")
print(transcribed_text)

Transcribed Text:
 شم قررتالعین یعنی نور چشم پادشه ها البته قررتالعین لغتاً یعنی سپیدی چشم لغتاً قرره یعنی سپیدی کسی که قریگ رولعین میشه اما عرب ها قررتالعین رو به معنای کسی که ما میگیم مایهی روشنی چشم هست نور چشم هست به کار میبودن مولانا بلا فاصله بعد از همین نقطه حالا میخواد حکمتهای رو برای ما بگوید از ره پنهان ز این این پسر میتشید آبی نخیل آن پدر پدر رو تشبیح میکند به نخل به درخت بلندقامتی که میوه دارد میوه های روتب شیرین دارد اما خود این درخت نخل از کجا تقضیه میکند اینجاست که مولوی برای اینکه باب دیگری رو برای ما بگو شاید بگید که از دو چشم پسر تقضیه می کرد این پدر یعنی همین که نگاهش به پسر می افتاد و نگاه پسر به پدر می افتاد این نخل ازامی خود و بارور می شد و رتبهای شیرین می داد چرا این رو می گه؟ یکی این که خب بدیهیست یه پدری وقتی که به فرزند برومند خودش نگاه می کنه شاد می شه که از شادیهای نادر است در این آن حالا چه پسرش باشه چه دخترش فرق نمی کنه و واقعا قضای روحی برای او فراهم می شود مخصوصا که فرزند شایسته باشد و به آدمی هزاران شادی می بخشد و هزاران بار آدمی شکر نعمت به جامعه این د

In [ ]:
# Save the output to a text file
output_file = "transcription_whisperv3.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(transcribed_text)

print(f"Transcription saved to {output_file}")

Transcription saved to transcription_whisperv3.txt


: 

### Whisper v3 - Fine Tuned

In [7]:
# Load the Persian Whisper model
pipe = pipeline("automatic-speech-recognition", model="MohammadKhosravi/whisper-large-v3-Persian")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.26it/s]
Device set to use cuda:0


In [8]:
# Run the speech-to-text pipeline
result = pipe(audio_file, return_timestamps=True)

/data/soroush/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [9]:
# Extract the transcribed text
transcribed_text = result["text"]

# Print the output
print("Transcribed Text:")
print(transcribed_text)

Transcribed Text:
شَمْ قُرّتالعین یعنی نور چشم بادشاه. البته قُرّتالعین لغتاً یعنی سپیدی چشم، لغتاً قُرّه یعنی سپیدی، یعنی کسی که قریه رولعین میشد.اما عربها قُرّتالعین رو به معنای کسی که ما میگیم مایهی روشنی چشم است، نور چشم است، به کار میبردند.مولانا، بلافاصله بعد از همین نکته، حالا میخواد حکمتهایی رو برای ما بگوید، از ره پنهان زعینین پسر میتشید آبی نخیل آن پدر، پدر را تشبیه میکند به نخل، یک درخت بلند قامتی که میوه دارد، میوههای رتب شیرین دارد، اما خود این درخت نخل از کجا تغذیه میکند؟اینجاست که مولوی برای اینکه باب دیگری رو به روی ما بگوشایند، بوید که از دو چشم پسر تغذیه میکرد، این پدر.به نهم اینکه نگاهش به پسر میافتاد و نگاه پسر به پدر میافتاد، این نخل عزا میخورد و بارور میشد و رتبهای شیرین میداد.چرا این را میگه؟ یکی اینکه، خب بدیهی است یه پدری وقتی که به فرزند برومند خودش نگاه میکنه شاد میشه که از شادیهای نادر است در این آنه، حالا چه پسرش باشه چه دخترش فرق نمیکنه، و واقعاً غذای روحی برای او فراهم میشود، مخصوصاً که فرزند شایسته باشد، خب به آدمی هزاران شادی میبخشد و هزاران بار آدمی شک

In [10]:
# Save the output to a text file
output_file = "transcription_whisperv3_ft.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(transcribed_text)

print(f"Transcription saved to {output_file}")

Transcription saved to transcription_whisperv3_ft.txt
